<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [1]:
import spacy
import wikipedia
import pandas as pd
from spacy import displacy

import urllib 
import time
import json
import requests 
from bs4 import BeautifulSoup

In [2]:
# read json file of wikipedia page title and summary
with open('../01_make_matching_list/title_summary.json', 'r') as f:
    DICT_PAGE_TITLE = json.load(f)

In [12]:
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

def find_wiki_title(term):
    title = wikipedia.search(term)
    if title:
        return title[0]

    
def find_wiki_summary(term):
    try:
        return wikipedia.summary(term)
    # if it is a ambiguous term, the function will return None as value of summary
    except wikipedia.exceptions.WikipediaException:
        return None


# def text_to_html(text, nlp): # gives the html under BeatifulSoup format
#     doc = nlp(text)

#     html = displacy.render(doc, style="ent", options={"ents": ["TERM"]}, jupyter=False, page=True)
#     soup = BeautifulSoup(html)
#     marks =  soup.find_all('mark')
    
#     for mark in tqdm(marks):
#         try:
#             term = mark.get_text(strip=True,separator=', ').split(', ')[0] # get the term annotated
            
#             wiki_info = DICT_PAGE_TITLE[term] # get wikipedia pagetitle and summary from json file
#             url = f'https://en.wikipedia.org/wiki/{"_".join(wiki_info["redirect"].split())}' 
#             summary = wiki_info['summary']
            
#         except KeyError:
#             wiki_title = find_wiki_title(term)  
            
#             if wiki_title:
#                 url = f'https://en.wikipedia.org/wiki/{"_".join(wiki_title.split())}' 
#                 wiki_summary = find_wiki_summary(wiki_title)
#                 DICT_PAGE_TITLE.update({term:{'title':wiki_title, 'summary': wiki_summary}})
                    
#         link = soup.new_tag('a', href=url) # create the html tag for link       
#         mark.wrap(link) #add html tag <a> (the one to make links) to around our annotated word
#     return soup


def text_to_json(text, nlp): # gives the ner results in json format
    from collections import defaultdict     

    dict_position = defaultdict(list)
    
    doc = nlp(text)
    for ent in doc.ents:
        dict_position[ent.text].append((ent.start_char, ent.end_char))
    dict_position = dict(dict_position)
    
    dict_res = {}
    for term, pos_l in tqdm(dict_position.items()):
        try:
            wiki_info = DICT_PAGE_TITLE[term] # get wikipedia pagetitle and summary from json file
            url = f'https://en.wikipedia.org/wiki/{"_".join(wiki_info["redirect"].split())}' 
            wiki_summary = wiki_info['summary'] 
            if wiki_summary is None:
                wiki_summary = find_wiki_summary(wiki_title)
                DICT_PAGE_TITLE[term]['summary'] = wiki_summary

        except KeyError:
            wiki_title = find_wiki_title(term)  

            if wiki_title:
                url = f'https://en.wikipedia.org/wiki/{"_".join(wiki_title.split())}' 
                wiki_summary = find_wiki_summary(wiki_title)
                DICT_PAGE_TITLE.update({term:{'title':wiki_title, 'summary': wiki_summary}})
            else:
                url = f'https://www.wikipedia.org/'
                wiki_summary = ''

        dict_res.update({term:{'position': pos_l,
                                'wikilink': url,
                                'summary': wiki_summary}})
    return dict_res    

In [4]:
nlp = spacy.load(r"./output/G06T_2015/model-best/") #load the best model

with open('../data/G06T0003000000.txt', encoding = 'utf-8') as f: # 24 patents inside
    patents = f.read().split('\n\n\n')

In [5]:
# open('./demo.html', 'w', encoding="utf-8").write(str(text_to_html(patents,nlp))) #create file with html data

In [13]:
# save new dictionary
with open('demo.json', "w") as f: 
    json.dump(text_to_json(patents[0], nlp), f, indent = 4)

100%|██████████| 80/80 [01:53<00:00,  1.42s/it]


In [8]:
# save new json 
with open('../01_make_matching_list/title_summary.json', "w") as f: 
    json.dump(DICT_PAGE_TITLE, f, indent = 4)